In [2]:
import pandas as pd



### load the data

In [3]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_target = pd.read_csv('sample_submission.csv')

In [4]:
df_train.isna().sum()


id             0
keyword       61
location    2533
text           0
target         0
dtype: int64

In [5]:
df_test.isna().sum()


id             0
keyword       26
location    1105
text           0
dtype: int64

In [6]:
df_target.isna().sum()


id        0
target    0
dtype: int64

In [7]:
df_train.keyword = df_train.keyword.fillna(df_train.keyword.mode()[0])
df_test.keyword = df_test.keyword.fillna(df_test.keyword.mode()[0])
df_train = df_train.dropna()
df_test = df_test.dropna()

In [8]:

df_train

,id,keyword,location,text,target
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0
...,...,...,...,...,...
7575,10826,wrecked,TN,On the bright side I wrecked http://t.co/uEa0t...,0
7577,10829,wrecked,#NewcastleuponTyne #UK,@widda16 ... He's gone. You can relax. I thoug...,0
7579,10831,wrecked,"Vancouver, Canada",Three days off from work and they've pretty mu...,0
7580,10832,wrecked,London,#FX #forex #trading Cramer: Iger's 3 words tha...,0


###  Text Clean 

In [9]:
# Libraries and packages for text (pre-)processing 
import string
import re
import nltk

In [10]:
df_train["clean_text"] = df_train["text"].apply(lambda x: x.lower())

In [11]:
!pip install contractions


[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: pip install --upgrade pip


In [12]:
import contractions

In [13]:
df_train["clean_text"] = df_train["clean_text"].apply(lambda x: contractions.fix(x))

In [14]:
df_train["clean_text"]

31      @bbcmtd wholesale markets ablaze http://t.co/l...
32      we always try to bring the heavy. #metal #rt h...
33      #africanbaze: breaking news:nigeria flag set a...
34                     crying out for more! set me ablaze
35      on plus side look at the sky last night it was...
                              ...                        
7575    on the bright side i wrecked http://t.co/uea0t...
7577    @widda16 ... he is gone. you can relax. i thou...
7579    three days off from work and they have pretty ...
7580    #fx #forex #trading cramer: iger's 3 words tha...
7581    @engineshed great atmosphere at the british li...
Name: clean_text, Length: 5080, dtype: object

In [15]:
print(df_train["text"][67])
print(df_train["clean_text"][67])

'I can't have kids cuz I got in a bicycle accident &amp; split my testicles. it's impossible for me to have kids' MICHAEL YOU ARE THE FATHER
'i cannot have kids cuz i got in a bicycle accident &amp; split my testicles. it is impossible for me to have kids' michael you are the father


In [17]:
def remove_URL(text):
    """
        Remove URLs from a sample string
    """
    return re.sub(r"https?://\S+|www\.\S+", "", text)



def remove_html(text):
    """
        Remove the html in sample text
    """
    html = re.compile(r"<.*?>|&([a-z0-9]+|#[0-9]{1,6}|#x[0-9a-f]{1,6});")
    return re.sub(html, "", text)


def remove_non_ascii(text):
    """
        Remove non-ASCII characters 
    """
    return re.sub(r'[^\x00-\x7f]',r'', text) # or ''.join([x for x in text if x in string.printable]) 

def remove_punct(text):
    """
        Remove the punctuation
    """
#     return re.sub(r'[]!"$%&\'()*+,./:;=#@?[\\^_`{|}~-]+', "", text)
    return text.translate(str.maketrans('', '', string.punctuation))

In [18]:
df_train["clean_text"] = df_train["clean_text"].apply(lambda x: remove_URL(x))

df_train["clean_text"] = df_train["clean_text"].apply(lambda x: remove_html(x))

df_train["clean_text"] = df_train["clean_text"].apply(lambda x: remove_non_ascii(x))

df_train["clean_text"] = df_train["clean_text"].apply(lambda x: remove_punct(x))

###  Text Preprocessing 

In [21]:
%time
# Tokenizing the tweet base texts.
from nltk.tokenize import word_tokenize

df_train['tokenized'] = df_train['clean_text'].apply(word_tokenize)
df_train.head()

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11.9 µs


,id,keyword,location,text,target,clean_text,tokenized
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze,"[bbcmtd, wholesale, markets, ablaze]"
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy metal rt,"[we, always, try, to, bring, the, heavy, metal..."
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...,"[africanbaze, breaking, newsnigeria, flag, set..."
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more set me ablaze,"[crying, out, for, more, set, me, ablaze]"
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...,"[on, plus, side, look, at, the, sky, last, nig..."


In [23]:
# Removing stopwords.
nltk.download("stopwords")
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
df_train['stopwords_removed'] = df_train['tokenized'].apply(lambda x: [word for word in x if word not in stop])
df_train.head()

[nltk_data] Downloading package stopwords to /home/aboud/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,id,keyword,location,text,target,clean_text,tokenized,stopwords_removed
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze,"[bbcmtd, wholesale, markets, ablaze]","[bbcmtd, wholesale, markets, ablaze]"
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy metal rt,"[we, always, try, to, bring, the, heavy, metal...","[always, try, bring, heavy, metal, rt]"
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...,"[africanbaze, breaking, newsnigeria, flag, set...","[africanbaze, breaking, newsnigeria, flag, set..."
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more set me ablaze,"[crying, out, for, more, set, me, ablaze]","[crying, set, ablaze]"
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...,"[on, plus, side, look, at, the, sky, last, nig...","[plus, side, look, sky, last, night, ablaze]"


In [24]:
from nltk.stem import PorterStemmer

def porter_stemmer(text):
    """
        Stem words in list of tokenized words with PorterStemmer
    """
    stemmer = nltk.PorterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems

In [25]:
%time 

df_train['porter_stemmer'] = df_train['stopwords_removed'].apply(lambda x: porter_stemmer(x))
df_train.head()



CPU times: user 10 µs, sys: 3 µs, total: 13 µs
Wall time: 25.7 µs


,id,keyword,location,text,target,clean_text,tokenized,stopwords_removed,porter_stemmer
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze,"[bbcmtd, wholesale, markets, ablaze]","[bbcmtd, wholesale, markets, ablaze]","[bbcmtd, wholesal, market, ablaz]"
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy metal rt,"[we, always, try, to, bring, the, heavy, metal...","[always, try, bring, heavy, metal, rt]","[alway, tri, bring, heavi, metal, rt]"
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...,"[africanbaze, breaking, newsnigeria, flag, set...","[africanbaze, breaking, newsnigeria, flag, set...","[africanbaz, break, newsnigeria, flag, set, ab..."
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more set me ablaze,"[crying, out, for, more, set, me, ablaze]","[crying, set, ablaze]","[cri, set, ablaz]"
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...,"[on, plus, side, look, at, the, sky, last, nig...","[plus, side, look, sky, last, night, ablaze]","[plu, side, look, sky, last, night, ablaz]"


In [26]:


from nltk.stem import SnowballStemmer

def snowball_stemmer(text):
    """
        Stem words in list of tokenized words with SnowballStemmer
    """
    stemmer = nltk.SnowballStemmer("english")
    stems = [stemmer.stem(i) for i in text]
    return stems



In [27]:



%time 

df_train['snowball_stemmer'] = df_train['stopwords_removed'].apply(lambda x: snowball_stemmer(x))
df_train.head()



CPU times: user 7 µs, sys: 2 µs, total: 9 µs
Wall time: 16.2 µs


,id,keyword,location,text,target,clean_text,tokenized,stopwords_removed,porter_stemmer,snowball_stemmer
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze,"[bbcmtd, wholesale, markets, ablaze]","[bbcmtd, wholesale, markets, ablaze]","[bbcmtd, wholesal, market, ablaz]","[bbcmtd, wholesal, market, ablaz]"
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy metal rt,"[we, always, try, to, bring, the, heavy, metal...","[always, try, bring, heavy, metal, rt]","[alway, tri, bring, heavi, metal, rt]","[alway, tri, bring, heavi, metal, rt]"
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...,"[africanbaze, breaking, newsnigeria, flag, set...","[africanbaze, breaking, newsnigeria, flag, set...","[africanbaz, break, newsnigeria, flag, set, ab...","[africanbaz, break, newsnigeria, flag, set, ab..."
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more set me ablaze,"[crying, out, for, more, set, me, ablaze]","[crying, set, ablaze]","[cri, set, ablaz]","[cri, set, ablaz]"
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...,"[on, plus, side, look, at, the, sky, last, nig...","[plus, side, look, sky, last, night, ablaze]","[plu, side, look, sky, last, night, ablaz]","[plus, side, look, sky, last, night, ablaz]"


In [28]:
from nltk.stem import LancasterStemmer

def lancaster_stemmer(text):
    """
        Stem words in list of tokenized words with LancasterStemmer
    """
    stemmer = nltk.LancasterStemmer()
    stems = [stemmer.stem(i) for i in text]
    return stems


In [30]:
%time 

df_train['lancaster_stemmer'] = df_train['stopwords_removed'].apply(lambda x: lancaster_stemmer(x))
df_train.head()

CPU times: user 8 µs, sys: 1e+03 ns, total: 9 µs
Wall time: 16.9 µs


,id,keyword,location,text,target,clean_text,tokenized,stopwords_removed,porter_stemmer,snowball_stemmer,lancaster_stemmer
31,48,ablaze,Birmingham,@bbcmtd Wholesale Markets ablaze http://t.co/l...,1,bbcmtd wholesale markets ablaze,"[bbcmtd, wholesale, markets, ablaze]","[bbcmtd, wholesale, markets, ablaze]","[bbcmtd, wholesal, market, ablaz]","[bbcmtd, wholesal, market, ablaz]","[bbcmtd, wholes, market, ablaz]"
32,49,ablaze,Est. September 2012 - Bristol,We always try to bring the heavy. #metal #RT h...,0,we always try to bring the heavy metal rt,"[we, always, try, to, bring, the, heavy, metal...","[always, try, bring, heavy, metal, rt]","[alway, tri, bring, heavi, metal, rt]","[alway, tri, bring, heavi, metal, rt]","[alway, try, bring, heavy, met, rt]"
33,50,ablaze,AFRICA,#AFRICANBAZE: Breaking news:Nigeria flag set a...,1,africanbaze breaking newsnigeria flag set abla...,"[africanbaze, breaking, newsnigeria, flag, set...","[africanbaze, breaking, newsnigeria, flag, set...","[africanbaz, break, newsnigeria, flag, set, ab...","[africanbaz, break, newsnigeria, flag, set, ab...","[africanbaz, break, newsniger, flag, set, abla..."
34,52,ablaze,"Philadelphia, PA",Crying out for more! Set me ablaze,0,crying out for more set me ablaze,"[crying, out, for, more, set, me, ablaze]","[crying, set, ablaze]","[cri, set, ablaz]","[cri, set, ablaz]","[cry, set, ablaz]"
35,53,ablaze,"London, UK",On plus side LOOK AT THE SKY LAST NIGHT IT WAS...,0,on plus side look at the sky last night it was...,"[on, plus, side, look, at, the, sky, last, nig...","[plus, side, look, sky, last, night, ablaze]","[plu, side, look, sky, last, night, ablaz]","[plus, side, look, sky, last, night, ablaz]","[plu, sid, look, sky, last, night, ablaz]"


In [33]:
nltk.download('brown')

from nltk.corpus import wordnet
from nltk.corpus import brown

wordnet_map = {"N":wordnet.NOUN, 
               "V":wordnet.VERB, 
               "J":wordnet.ADJ, 
               "R":wordnet.ADV
              }
    
train_sents = brown.tagged_sents(categories='news')
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)

def pos_tag_wordnet(text, pos_tag_type="pos_tag"):
    """
        Create pos_tag with wordnet format
    """
    pos_tagged_text = t2.tag(text)
    
    # map the pos tagging output with wordnet output 
    pos_tagged_text = [(word, wordnet_map.get(pos_tag[0])) if pos_tag[0] in wordnet_map.keys() else (word, wordnet.NOUN) for (word, pos_tag) in pos_tagged_text ]
    return pos_tagged_text

[nltk_data] Downloading package brown to /home/aboud/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
